In [1]:
import torch
# GPU 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name(0))

# CUDA 사용 가능 여부 확인
print(f"CUDA available: {torch.cuda.is_available()}")  # True여야 함

# PyTorch에서 사용하는 CUDA 버전 확인
print(f"CUDA version in PyTorch: {torch.version.cuda}")

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cuda
NVIDIA GeForce RTX 4050 Laptop GPU
CUDA available: True
CUDA version in PyTorch: 11.8


In [2]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
import matplotlib.pyplot as plt
import torch.nn.functional as F

d:\sy\test\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
from torchvision import datasets
import pandas as pd

IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 10

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
])

# 데이터셋 디렉토리 설정
train_dir = '../PCB_imgs/all/resize/train'
val_dir = '../PCB_imgs/all/resize/validation'
test_dir = '../PCB_imgs/all/resize/test'

# ImageFolder로 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 파일 경로 및 타겟 추출
train_file_paths = [img[0] for img in train_dataset.imgs]
train_targets = train_dataset.targets

val_file_paths = [img[0] for img in val_dataset.imgs]
val_targets = val_dataset.targets

test_file_paths = [img[0] for img in test_dataset.imgs]
test_targets = test_dataset.targets

# DataFrame 생성
train_df = pd.DataFrame({'file_paths': train_file_paths, 'targets': train_targets})
validation_df = pd.DataFrame({'file_paths': val_file_paths, 'targets': val_targets})
test_df = pd.DataFrame({'file_paths': test_file_paths, 'targets': test_targets})

# 확인을 위해 각 데이터셋의 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Validation 데이터 수: {len(validation_df)}")
print(f"Test 데이터 수: {len(test_df)}")

Train 데이터 수: 2008
Validation 데이터 수: 502
Test 데이터 수: 628


In [4]:
import torch
from torchvision import datasets, transforms

# resize된 데이터셋 로드
input_dir = '../PCB_imgs/all/resize/'
dataset = datasets.ImageFolder(root=input_dir, transform=transform)

# 데이터 로더 정의
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 데이터 로더에서 데이터 가져오기
for images, labels in dataloader:
    print(images.shape)  # 배치의 이미지 텐서 크기 확인
    break  # 첫 번째 배치만 확인
    # 배치 크기, 채널 수(RGB), 이미지 크기 

torch.Size([32, 3, 256, 256])


In [5]:
IMAGE_SIZE = 299
BATCH_SIZE = 32
EPOCHS = 10

# 커스텀 데이터세트 정의
class CustomDataset(Dataset):
    def __init__(self, file_paths, targets, aug=None, preprocess=None):
        self.file_paths = file_paths
        self.targets = targets
        self.aug = aug
        self.preprocess = preprocess

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        target = self.targets[index]
        
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        if self.aug is not None:
            image = self.aug(image=image)['image']

        if self.preprocess is not None:
            image = self.preprocess(image)

        image = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        
        return image, target

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [14]:
# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='vgg16'):
        super(CustomModel, self).__init__()
        if model_name == 'vgg16':
            self.base_model = torchvision.models.vgg16(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'mobilenet':
            self.base_model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.fc2 = nn.Linear(50, 1)  # Sigmoid 출력

    def _get_features_dim(self, model_name):
        if model_name in 'vgg16':
            return 25088  # VGG16의 출력 차원
        elif model_name in ['resnet50', 'inception']:
            return 2048  # ResNet50, inception의 출력 차원
        elif model_name == 'mobilenet':
            return 1280  # MobileNetV2의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Sigmoid 제거
        # x = torch.sigmoid(self.fc2(x))
        return x

In [7]:
# 평가 함수 정의
def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            outputs = outputs.view(-1) # torch.Size([batch_size])로 변환
            loss = criterion(outputs, labels)  # 손실 계산
            running_test_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # 0.5 기준으로 이진 분류
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
            
    # 검증 손실과 정확도
    test_loss = running_test_loss / len(test_loader)
    test_accuracy = correct_test / total_test
    return test_loss, test_accuracy

##### Inception 모델 구조만 이용

In [13]:
import torch
import torch.nn as nn
import torchvision

class CustomModel(nn.Module):
    def __init__(self, model_name='vgg16'):
        super(CustomModel, self).__init__()
        if model_name == 'vgg16':
            self.base_model = torchvision.models.vgg16()
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50()
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3()
            self.base_model.fc = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'mobilenet':
            self.base_model = torchvision.models.mobilenet_v2()
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)

    def _get_features_dim(self, model_name):
        if model_name == 'vgg16':
            return 25088  # VGG16의 출력 차원
        elif model_name in ['resnet50', 'inception']:
            return 2048  # ResNet50 및 Inception의 출력 차원
        elif model_name == 'mobilenet':
            return 1280  # MobileNetV2의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        # Inception 모델에 대한 수정
        if isinstance(x, tuple):  # Inception 모델이 여러 출력을 반환하는 경우
            x = x[0]  # 첫 번째 출력을 선택
        
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)  # Sigmoid 제거
        return x  # 최종 출력


In [14]:
# 모델 초기화
model = CustomModel(model_name='inception').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

d:\sy\test\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [15]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장 코드를 추가할 수 있습니다.
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.5306, Train Accuracy: 0.7405, Val Loss: 0.3859, Val Accuracy: 0.8108
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.3745, Train Accuracy: 0.8162, Val Loss: 0.3444, Val Accuracy: 0.8486
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.3165, Train Accuracy: 0.8566, Val Loss: 0.3924, Val Accuracy: 0.8446
Epoch [4/20], Train Loss: 0.3068, Train Accuracy: 0.8710, Val Loss: 0.3811, Val Accuracy: 0.8566
Epoch [5/20], Train Loss: 0.2805, Train Accuracy: 0.8790, Val Loss: 0.3162, Val Accuracy: 0.8606
Validation loss improved, saving model...
Epoch [6/20], Train Loss: 0.2436, Train Accuracy: 0.8969, Val Loss: 0.3729, Val Accuracy: 0.8247
Epoch [7/20], Train Loss: 0.2156, Train Accuracy: 0.9109, Val Loss: 0.3913, Val Accuracy: 0.8645
Epoch [8/20], Train Loss: 0.2102, Train Accuracy: 0.9218, Val Loss: 0.3595, Val Accuracy: 0.8725
Epoch [9/20], Train Loss: 0.1957, Train Accuracy: 0.9218, Val Loss: 0.2952, Val Accuracy: 0.8884
V

17 epoch -> 6m 49.3s  
1 epoch -> 24s

In [16]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.2606, Test Accuracy: 0.9124


16 epoch에서 조기 종료 발생.  
훈련데이터의 손실값은 감소하는 반면 검증 데이터의 손실값은 증감을 반복함. -> 과적합   

Imagenet 데이터에 대해 미리 학습된 파라미터 값을 이용하여 훈련 재진행. 

##### Inception모델 구조 + ImageNet 데이터에 대해 미리 학습된 파라미터값 사용

In [8]:
import torch
import torch.nn as nn
import torchvision

class CustomModel(nn.Module):
    def __init__(self, model_name='vgg16'):
        super(CustomModel, self).__init__()
        if model_name == 'vgg16':
            self.base_model = torchvision.models.vgg16(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.fc = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'mobilenet':
            self.base_model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)

    def _get_features_dim(self, model_name):
        if model_name == 'vgg16':
            return 25088  # VGG16의 출력 차원
        elif model_name in ['resnet50', 'inception']:
            return 2048  # ResNet50 및 Inception의 출력 차원
        elif model_name == 'mobilenet':
            return 1280  # MobileNetV2의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        # Inception 모델에 대한 수정
        if isinstance(x, tuple):  # Inception 모델이 여러 출력을 반환하는 경우
            x = x[0]  # 첫 번째 출력을 선택
        
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)  # Sigmoid 제거
        return x  # 최종 출력


In [9]:
# 모델 초기화
model = CustomModel(model_name='inception').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [10]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장 코드를 추가할 수 있습니다.
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.3435, Train Accuracy: 0.8471, Val Loss: 0.2101, Val Accuracy: 0.9203
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.1041, Train Accuracy: 0.9686, Val Loss: 0.1758, Val Accuracy: 0.9442
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.0524, Train Accuracy: 0.9846, Val Loss: 0.1694, Val Accuracy: 0.9382
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.0374, Train Accuracy: 0.9880, Val Loss: 0.1787, Val Accuracy: 0.9522
Epoch [5/20], Train Loss: 0.0590, Train Accuracy: 0.9781, Val Loss: 0.1907, Val Accuracy: 0.9402
Epoch [6/20], Train Loss: 0.0342, Train Accuracy: 0.9880, Val Loss: 0.1503, Val Accuracy: 0.9482
Validation loss improved, saving model...
Epoch [7/20], Train Loss: 0.0128, Train Accuracy: 0.9985, Val Loss: 0.1623, Val Accuracy: 0.9562
Epoch [8/20], Train Loss: 0.0026, Train Accuracy: 1.0000, Val Loss: 0.1500, Val Accuracy: 0.9701
Validation loss improved, saving model...
Epoch [9/20], 

14 epoch -> 5m 33.2s  
1 epoch -> 24s

In [11]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.1733, Test Accuracy: 0.9554


훈련데이터와 검증데이터의 정확도 모두 상승하긴 하였으나, 여전히 과적합의 양상을 보임.  

##### Inception 모델 + ImageNet 파라미터 + 데이터 증강

In [18]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
# 모델 초기화
model = CustomModel(model_name='inception').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

d:\sy\test\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [20]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/inception_best_model.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

Epoch [1/20], Train Loss: 0.5783, Train Accuracy: 0.6962, Val Loss: 0.4680, Val Accuracy: 0.7789
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.4587, Train Accuracy: 0.7789, Val Loss: 0.3982, Val Accuracy: 0.8048
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.4265, Train Accuracy: 0.7854, Val Loss: 0.3833, Val Accuracy: 0.8327
Validation loss improved, saving model...
Epoch [4/20], Train Loss: 0.4052, Train Accuracy: 0.8073, Val Loss: 0.3347, Val Accuracy: 0.8466
Validation loss improved, saving model...
Epoch [5/20], Train Loss: 0.4094, Train Accuracy: 0.8108, Val Loss: 0.3469, Val Accuracy: 0.8446
Epoch [6/20], Train Loss: 0.3906, Train Accuracy: 0.8078, Val Loss: 0.3360, Val Accuracy: 0.8645
Epoch [7/20], Train Loss: 0.4035, Train Accuracy: 0.8187, Val Loss: 0.3207, Val Accuracy: 0.8606
Validation loss improved, saving model...
Epoch [8/20], Train Loss: 0.3785, Train Accuracy: 0.8237, Val Loss: 0.3795, Val Accuracy: 0.8127
Epoch [9/20], 

20 epoch -> 8m 49.3s  
1 epoch -> 26s

In [25]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='inception').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/inception_best_model.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

d:\sy\test\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
C:\Users\enssel\AppData\Local\Temp\ipykernel_21184\3232790612.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly a

CustomModel(
  (base_model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): 

In [26]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.2466, Test Accuracy: 0.8869


과적합은 없지만 모델의 정확도가 0.86정도로 높지않고, 손실값이 0.3정도로 높음.   
충분한 학습을 위해 50 epoch로 학습 시간을 늘리고 학습률 스케줄러를 함께 사용해보기.  

##### Inception모델 + ImageNet 파라미터 + 증강 (50 epoch + early stopping + 학습률 스케줄러)

In [31]:
# 모델 초기화
model = CustomModel(model_name='inception').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

d:\sy\test\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [32]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 10  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")  
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/inception_best_model_02.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.5889, Train Accuracy: 0.6823, Val Loss: 0.4524, Val Accuracy: 0.8008
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.4746, Train Accuracy: 0.7694, Val Loss: 0.4289, Val Accuracy: 0.7908
Epoch [3/50], Train Loss: 0.4680, Train Accuracy: 0.7694, Val Loss: 0.5407, Val Accuracy: 0.7869
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.4248, Train Accuracy: 0.7859, Val Loss: 0.4159, Val Accuracy: 0.7888
Validation loss improved, saving model...
Epoch [5/50], Train Loss: 0.4246, Train Accuracy: 0.7928, Val Loss: 0.3873, Val Accuracy: 0.8227
Validation loss improved, saving model...
Epoch [6/50], Train Loss: 0.4153, Train Accuracy: 0.7963, Val Loss: 0.3442, Val Accuracy: 0.8486
Validation loss improved, saving model...
Epoch [7/50], Train Loss: 0.3917, Train Accuracy: 0.8132, Val Loss: 0.3434, Val Accuracy: 0.8606
Epoch [8/50], Train Loss: 0.3924, Train Accuracy: 0.8207, Val Loss: 0

50 epoch -> 22m 0.9s     
1 epoch -> 26s

In [33]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='inception').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/inception_best_model_02.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_21184\3483221002.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/inception_best_model_02.p

CustomModel(
  (base_model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): 

In [34]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.1846, Test Accuracy: 0.9172


Train: 0.93 / 0.17
Val: 0.93 / 0.19  
Test: 0.91 / 0.18  
-> 과적합없이 0.9정도의 정확도를 보이는 모델이 학습됨.  
이전보다 모델의 성능은 올랐지만, 조금 더 올려보기 위해 batch size를 16으로 줄이고, 데이터의 특징을 뽑아 2048 차원을 50차원으로 축소하는 층을 추가해보고자한다.

In [35]:
from torchvision import datasets
import pandas as pd

IMAGE_SIZE = 299
BATCH_SIZE = 16
EPOCHS = 10

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
])

# 데이터셋 디렉토리 설정
train_dir = '../PCB_imgs/all/resize/train'
val_dir = '../PCB_imgs/all/resize/validation'
test_dir = '../PCB_imgs/all/resize/test'

# ImageFolder로 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 파일 경로 및 타겟 추출
train_file_paths = [img[0] for img in train_dataset.imgs]
train_targets = train_dataset.targets

val_file_paths = [img[0] for img in val_dataset.imgs]
val_targets = val_dataset.targets

test_file_paths = [img[0] for img in test_dataset.imgs]
test_targets = test_dataset.targets

# DataFrame 생성
train_df = pd.DataFrame({'file_paths': train_file_paths, 'targets': train_targets})
validation_df = pd.DataFrame({'file_paths': val_file_paths, 'targets': val_targets})
test_df = pd.DataFrame({'file_paths': test_file_paths, 'targets': test_targets})

# 확인을 위해 각 데이터셋의 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Validation 데이터 수: {len(validation_df)}")
print(f"Test 데이터 수: {len(test_df)}")

Train 데이터 수: 2008
Validation 데이터 수: 502
Test 데이터 수: 628


In [36]:
import torch
from torchvision import datasets, transforms

# resize된 데이터셋 로드
input_dir = '../PCB_imgs/all/resize/'
dataset = datasets.ImageFolder(root=input_dir, transform=transform)

# 데이터 로더 정의
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 데이터 로더에서 데이터 가져오기
for images, labels in dataloader:
    print(images.shape)  # 배치의 이미지 텐서 크기 확인
    break  # 첫 번째 배치만 확인
    # 배치 크기, 채널 수(RGB), 이미지 크기 

torch.Size([16, 3, 299, 299])


##### Inception + ImageNet + 데이터 증강 (batch size 16, 2048 -> 50차원 축소, 50 epoch, early stopping 10, 학습률 스케줄러)

In [49]:
import torch
import torch.nn as nn
import torchvision

class CustomModel(nn.Module):
    def __init__(self, model_name='vgg16'):
        super(CustomModel, self).__init__()
        if model_name == 'vgg16':
            self.base_model = torchvision.models.vgg16(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.fc = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'mobilenet':
            self.base_model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.fc2 = nn.Linear(50, 1)  # Sigmoid 출력

    def _get_features_dim(self, model_name):
        if model_name == 'vgg16':
            return 25088  # VGG16의 출력 차원
        elif model_name in ['resnet50', 'inception']:
            return 2048  # ResNet50 및 Inception의 출력 차원
        elif model_name == 'mobilenet':
            return 1280  # MobileNetV2의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        # Inception 모델에 대한 수정
        if isinstance(x, tuple):  # Inception 모델이 여러 출력을 반환하는 경우
            x = x[0]  # 첫 번째 출력을 선택
        
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.fc2(x)
        return x  # 최종 출력

In [50]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [51]:
# 모델 초기화
model = CustomModel(model_name='inception').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [52]:
EPOCHS = 50

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 10  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")  
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/inception_best_model_03.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/50], Train Loss: 0.4041, Train Accuracy: 0.8113, Val Loss: 0.2841, Val Accuracy: 0.8904
Validation loss improved, saving model...
Epoch [2/50], Train Loss: 0.2383, Train Accuracy: 0.9014, Val Loss: 0.1461, Val Accuracy: 0.9502
Epoch [3/50], Train Loss: 0.1089, Train Accuracy: 0.9617, Val Loss: 0.1581, Val Accuracy: 0.9442
Validation loss improved, saving model...
Epoch [4/50], Train Loss: 0.0937, Train Accuracy: 0.9656, Val Loss: 0.1076, Val Accuracy: 0.9721
Epoch [5/50], Train Loss: 0.0680, Train Accuracy: 0.9776, Val Loss: 0.1152, Val Accuracy: 0.9721
Epoch [6/50], Train Loss: 0.0640, Train Accuracy: 0.9791, Val Loss: 0.1379, Val Accuracy: 0.9681
Epoch [7/50], Train Loss: 0.0507, Train Accuracy: 0.9851, Val Loss: 0.1384, Val Accuracy: 0.9741
Epoch [8/50], Train Loss: 0.0504, Train Accuracy: 0.9861, Val Loss: 0.1423, Val Accuracy: 0.9781
Epoch [9/50], Train Loss: 0.0301, Train Accuracy: 0.9900, Val Loss: 0.1146, Val Accuracy: 0.9801
E

34 epoch -> 15m 16.8s  
1 epoch -> 27s

In [53]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='inception').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/inception_best_model_03.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_21184\100461600.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/inception_best_model_03.pt

CustomModel(
  (base_model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): 

In [54]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0770, Test Accuracy: 0.9809


훈련데이터의 손실값은 계속 감소하지만, 검증데이터의 손실값은 10 epoch이후로 거의 감소하지 않음.   
또한 훈련 데이터의 정확도가 1.0을 보임.  

과적합 양상을 보이므로 Dropout 층 추가해보기.

##### Inception + ImageNet + 데이터 증강 (batch size 16, 2048 -> 50차원 축소, Dropout, 20 epoch, early stopping(10 -> 3), 학습률 스케줄러)

In [72]:
import torch
import torch.nn as nn
import torchvision

class CustomModel(nn.Module):
    def __init__(self, model_name='vgg16'):
        super(CustomModel, self).__init__()
        if model_name == 'vgg16':
            self.base_model = torchvision.models.vgg16(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
            self.base_model.fc = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'mobilenet':
            self.base_model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V1')
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self._get_features_dim(model_name), 50)
        self.fc2 = nn.Linear(50, 1)  # Sigmoid 출력

    def _get_features_dim(self, model_name):
        if model_name == 'vgg16':
            return 25088  # VGG16의 출력 차원
        elif model_name in ['resnet50', 'inception']:
            return 2048  # ResNet50 및 Inception의 출력 차원
        elif model_name == 'mobilenet':
            return 1280  # MobileNetV2의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        # Inception 모델에 대한 수정
        if isinstance(x, tuple):  # Inception 모델이 여러 출력을 반환하는 경우
            x = x[0]  # 첫 번째 출력을 선택
        
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x  # 최종 출력

In [73]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터 증강 정의
aug = A.Compose([
    A.HorizontalFlip(p=0.5),                # 좌우 반전
    A.VerticalFlip(p=0.5),                  # 상하 반전
    A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
    A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
])

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values, aug=aug)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [74]:
# 모델 초기화
model = CustomModel(model_name='inception').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [75]:
EPOCHS = 20

# 조기 종료 변수 초기화
best_val_loss = float('inf')
patience = 3  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 학습률 스케줄러
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    # 학습률 스케줄러 적용
    scheduler.step(val_loss)

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")  
        # 모델 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, '../model/inception_best_model_04.pth')
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation loss improved, saving model...
Epoch [1/20], Train Loss: 0.4611, Train Accuracy: 0.7759, Val Loss: 0.2896, Val Accuracy: 0.8845
Validation loss improved, saving model...
Epoch [2/20], Train Loss: 0.2696, Train Accuracy: 0.8860, Val Loss: 0.1911, Val Accuracy: 0.9263
Validation loss improved, saving model...
Epoch [3/20], Train Loss: 0.1619, Train Accuracy: 0.9323, Val Loss: 0.1366, Val Accuracy: 0.9402
Epoch [4/20], Train Loss: 0.1015, Train Accuracy: 0.9636, Val Loss: 0.2186, Val Accuracy: 0.9263
Validation loss improved, saving model...
Epoch [5/20], Train Loss: 0.0944, Train Accuracy: 0.9731, Val Loss: 0.0966, Val Accuracy: 0.9761
Validation loss improved, saving model...
Epoch [6/20], Train Loss: 0.0512, Train Accuracy: 0.9851, Val Loss: 0.0738, Val Accuracy: 0.9781
Epoch [7/20], Train Loss: 0.0487, Train Accuracy: 0.9851, Val Loss: 0.1247, Val Accuracy: 0.9781
Epoch [8/20], Train Loss: 0.0584, Train Accuracy: 0.9841, Val Loss: 0.1336, Val Accuracy: 0.9701
Early stopping

11 epoch -> 5m 23.3s  
1 epoch -> 30s

In [76]:
# 모델 클래스 정의 (CustomModel)
model = CustomModel(model_name='inception').to(DEVICE)

# 저장된 체크포인트 불러오기
checkpoint = torch.load('../model/inception_best_model_04.pth')

# 모델에 체크포인트 적용
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

# 모델을 평가 모드로 전환
model.eval()

C:\Users\enssel\AppData\Local\Temp\ipykernel_21184\2502014020.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../model/inception_best_model_04.p

CustomModel(
  (base_model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): 

In [77]:
# 테스트 데이터세트로 평가
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0721, Test Accuracy: 0.9809


Train: 0.98 / 0.05  
Val: 0.98 / 0.07  
Test: 0.98 / 0.07  
해당 모델이 best 모델임.  
더 이상 학습을 진행하면 과적합이 심해질 것이라 판단하여 중지.

##### KFold 교차검증으로 과적합 재확인

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A

# 데이터셋 디렉토리 설정
data_dir = '../PCB_imgs/all/resize/all'  # 모든 이미지를 포함하는 디렉토리

# 이미지와 레이블 로드
def load_data(data_dir):
    file_paths = []
    targets = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for img_file in os.listdir(label_dir):
            file_paths.append(os.path.join(label_dir, img_file))
            targets.append(label)
    return pd.DataFrame({'file_paths': file_paths, 'targets': targets})

# 전체 데이터셋 로드
full_dataset = load_data(data_dir)

# 전체 데이터셋에서 Train (80%)과 Validation (20%) 나누기
train_df, val_df = train_test_split(full_dataset, test_size=0.2, random_state=124)

# Train, Validation 데이터 개수 확인
print(f'Train 데이터 수: {len(train_df)}')     
print(f'Validation 데이터 수: {len(val_df)}') 

# k-Fold Cross Validation 설정
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=124)

# 성능 저장을 위한 리스트 초기화
fold_train_losses = []
fold_val_losses = []
fold_train_accuracies = []
fold_val_accuracies = []

# 5번 k-Fold Cross Validation 진행
for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
    print(f'Fold {fold + 1}/{k_folds}')
    
    # 각 fold에 대해 데이터셋 설정
    train_subset = train_df.iloc[train_idx]
    validation_subset = train_df.iloc[val_idx]

    aug = A.Compose([
        A.HorizontalFlip(p=0.5),                # 좌우 반전
        A.VerticalFlip(p=0.5),                  # 상하 반전
        A.Rotate(limit=10, p=0.5),              # 작은 각도 회전 (10도 내외)
        A.RandomBrightnessContrast(p=0.5),      # 밝기 및 대비 조절
    ])

    # 이미지 변환 정의
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
    ])

    # 데이터셋 인스턴스 생성
    train_dataset = CustomDataset(train_subset['file_paths'].values, train_subset['targets'].values, aug=aug, preprocess=transform)
    validation_dataset = CustomDataset(validation_subset['file_paths'].values, validation_subset['targets'].values, preprocess=transform)

    # DataLoader 설정
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 모델 초기화
    model = CustomModel(model_name='inception').to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    criterion = nn.BCEWithLogitsLoss()
    
    best_val_loss = float('inf')
    patience_counter = 0

    # 각 fold에 대해 학습 진행
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        for images, targets in train_loader:
            images, targets = images.to(DEVICE), targets.to(DEVICE).float()
            
            optimizer.zero_grad()
            outputs = model(images)
            
            # 손실 계산
            loss = criterion(outputs.view(-1), targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # 정확도 계산
            predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
            correct_predictions += (predicted == targets).sum().item()
            total_predictions += targets.size(0)
        
        train_accuracy = correct_predictions / total_predictions
        fold_train_losses.append(running_loss / len(train_loader))
        fold_train_accuracies.append(train_accuracy)

        # 검증
        model.eval()
        val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)
        fold_val_losses.append(val_loss)
        fold_val_accuracies.append(val_accuracy)
        
        # 조기 종료 로직
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # 개선 시 카운터 리셋
        else:
            patience_counter += 1  # 개선 없을 시 카운터 증가
        
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
    
    print(f'Fold {fold + 1} completed. Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

# 최종 성능 요약
print("Training completed across all folds.")




##### fine tuning